In [1]:
import os
import os.path as osp
import sys
import logging
import csv
import io
import base64
import ast

from tqdm import tqdm
import json


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas  
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator



from flask import Flask, jsonify, request, render_template
from flask_cors import CORS 
from ragutils import ChatGpt, ragChat
from langchain_openai import ChatOpenAI
import importlib
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document




In [2]:
modelDict = dict()
with open ('./shapetalk.csv', 'r', encoding='gbk') as f:
    counter = 0
    print('initializing')
    reader = csv.reader(f)
    for row in tqdm(reader):
        if (row[6] not in modelDict):
            modelDict[row[6]] = set()
        for i in range(1, 5):
            if (row[i] != ''):
                modelDict[row[6]].add(row[i])
        counter += 1

modelDict.pop('ShapeNet')
modelDict.pop('source_object_class')
print (modelDict.keys())

vaseDocs = []
vaseId = []
i = 0
# for sentence in list(modelDict['vase']):
#     vaseDocs.append(Document(page_content=sentence))
#     vaseId.append('vase' + str(i))
#     i += 1
source_knowledge_dict = {}
for key in modelDict.keys():
    print (key + '-' + str(len(list(modelDict[key]))))
    source_knowledge_dict[key] = []
    sentenCounter = 0
    longSentences = ''
    for sentence in modelDict[key]:
        if (sentenCounter == 100):
            source_knowledge_dict[key].append(longSentences)
            longSentences = ''
            sentenCounter = 0
        longSentences += sentence + '\n'
        sentenCounter += 1
    if (sentenCounter > 0):
        source_knowledge_dict[key].append(longSentences)
        longSentences = ''
        sentenCounter = 0
# print (source_knowledge_dict)
# source_class = 'cap'
# for sentence in modelDict['cap']:
#     source_knowledge_cap += sentence + '\n'
# print(source_knowledge_cap)

initializing


130343it [00:01, 122540.45it/s]


dict_keys(['faucet', 'cap', 'plant', 'cabinet', 'bookshelf', 'knife', 'table', 'skateboard', 'mug', 'vase', 'pistol', 'flowerpot', 'clock', 'chair', 'bathtub', 'bottle', 'display', 'bag', 'trashbin', 'scissors', 'person', 'helmet', 'bowl', 'airplane', 'guitar', 'dresser', 'bed', 'sofa', 'bench', 'lamp'])
faucet-7041
cap-2094
plant-2687
cabinet-2578
bookshelf-8247
knife-4915
table-46480
skateboard-1980
mug-1491
vase-7016
pistol-3660
flowerpot-5625
clock-6116
chair-39479
bathtub-8594
bottle-4149
display-8047
bag-2726
trashbin-4056
scissors-715
person-1768
helmet-1451
bowl-1852
airplane-24645
guitar-6823
dresser-12808
bed-6860
sofa-29684
bench-14045
lamp-36758


In [4]:
# 这里会消耗巨量tokens，慎用
# embed_model = OpenAIEmbeddings(base_url="https://api3.apifans.com/v1")
# vectorstore = Chroma.from_documents(documents=vaseDocs, embedding=embed_model, collection_name="model_discription_embed")
# print (vectorstore)

In [14]:
query = "How can I change the vase?"
result = vectorstore.similarity_search(query ,k = 5)
result

NameError: name 'vectorstore' is not defined

In [3]:


for key in source_knowledge_dict.keys():
    print (len(source_knowledge_dict[key]))

71
21
27
26
83
50
465
20
15
71
37
57
62
395
86
42
81
28
41
8
18
15
19
247
69
129
69
297
141
368


In [26]:
ragClient = ragChat()
source_knowledge = 'The target\'s foot is more round and the vase is with builtiful color, the top of it has a longger head\nThe top of the vase is open\nThe vase has flowers in it'
modelClass = 'vase'
prompt_template1 = f"""以下内容为对一个{modelClass}的描述集合，将以下内容中的每一句按照格式以键值字典的方式拆分，键为该物体的部位，如果是对整体的描述则直接用{modelClass}作为键，值为该部位对应的描述。请返回给我一个键值字典列表，列表中每一项为一个键值字典。不要使用代码格式返回:

内容:
{source_knowledge}

"""
ragClient.pushUserMessage(prompt_template1)

# 以下操作消耗大量tokens
# answer = ragClient.getGptMsg()
# print (answer)

当前进度：10/10

In [30]:
# ragClient = ragChat()

# dict_keys(['faucet', 'cap', 'plant', 'cabinet', 'bookshelf', 'knife', 'table', 
# 'skateboard', 'mug', 'vase', 'pistol', 'flowerpot', 'clock', 'chair', 'bathtub', 
# 'bottle', 'display', 'bag', 'trashbin', 'scissors', 'person', 'helmet', 'bowl', 
# 'airplane', 'guitar', 'dresser', 'bed', 'sofa', 'bench', 'lamp'])
part2Discription = {}
for modelClass in source_knowledge_dict.keys():
    # 先不处理长度大于一百万的句子，tokens撑不住
# modelClass = 'faucet'
    # if (len(source_knowledge_dict[modelClass]) < 10):
    discription = dict()
    index = 0
    for sentences in source_knowledge_dict[modelClass]:
        if (index > 80):
            break
        print(f"\r当前进度：{modelClass}-{index}", end='')
        index += 1
        prompt_template1 = f"""以下内容为对一个{modelClass}的描述集合，将以下内容中的每一句按照格式以键值字典的方式拆分，键为该物体的部位，如果是对整体的描述则直接用{modelClass}作为键，值为该部位对应的描述。请返回给我一个python键值字典列表，列表中每一项为一个键值字典。不要使用代码格式返回:
    
        内容:
        {sentences}
        """
        ragClient.pushUserMessage(prompt_template1)
        answer = ragClient.getGptMsg()
        parsedStr = json.loads(answer)
        for dictEle in parsedStr:
            for key in dictEle.keys():
                if key in discription:
                    discription[key].add(dictEle[key])
                else:
                    discription[key] = set()
                    discription[key].add(dictEle[key])
    for key in discription.keys():
        discription[key] = list(discription[key])
    part2Discription[modelClass] = discription


当前进度：faucet-4

InternalServerError: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.22.1</center>
</body>
</html>

In [17]:
print (part2Discription.keys())

dict_keys([])


In [53]:
# answer = '''[
#   {"foot": "more round"},
#   {"vase": "with beautiful color"},
#   {"top": "has a longer head"},
#   {"top": "is open"},
#   {"vase": "has flowers in it"},
#   {"top": "has a longer head"}]'''
# answer += ''
# print (answer)
# print (answer)
parsedStr = json.loads(answer)

vasePart2Discription = {}
for dictEle in parsedStr:
    for key in dictEle.keys():
        if key in vasePart2Discription:
            vasePart2Discription[key].add(dictEle[key])
        else:
            vasePart2Discription[key] = set()
            vasePart2Discription[key].add(dictEle[key])
# print (vasePart2Discription)
for key in vasePart2Discription.keys():
    vasePart2Discription[key] = list(vasePart2Discription[key])
with open(source_class + '_Part2Discription', 'w') as f:
    json.dump(vasePart2Discription, f, indent=4)

In [54]:
# 1：这个是基于用户问答做反馈
query = '请问我可以对这个物品的那些部位做改动？'
prompt_template2 = f"""请基于以下内容回答问题

内容:
{vasePart2Discription}
query:
{query}
"""
ragClient.pushUserMessage(prompt_template2)
answer = ragClient.getGptMsg()
print (answer)

# 2：对用户输入prompt进行编码，做embedding相似的编辑方向推荐

根据所提供的描述，你可以对该物品的以下部位做改动：

1. **帽檐（brim）**：
   - 改变形状，例如将帽檐从圆形改为方形，或调整帽檐的宽度和厚度。
   - 调整帽檐的弯曲度，例如使其更平坦或更弯曲。
   - 增加或减少帽檐的长度或高度。

2. **帽舌（visor）**：
   - 改变帽舌的形状，例如从圆形改为方形或其他几何形状。
   - 调整帽舌的厚度或长度。
   - 改变帽舌的弯曲度或位置。

3. **顶部（top）**：
   - 改变顶部的形状，例如从尖顶改为平顶或圆顶。
   - 调整顶部的直径或高度。
   - 添加或移除顶部的饰物或标志。

4. **边缘（rim）**：
   - 调整边缘的高度或厚度。
   - 改变边缘的形状或弯曲度。

5. **按扣（button）**：
   - 改变按扣的大小或形状。
   - 添加或移除按扣。

这些改动可以根据个人喜好和风格需求进行调整，从而改变帽子的外观和功能。
